# Notebook Iteracion 4

 Modelado

### Objetivo:
Implementar estrategias de ensamble de modelos (boosting) con el fin de evaluar si estrategias mas robustas otorgan resultados acordes a los requisitos del negocio (MAPE <= 0.15):
- Ada Boosting
- Gradient Boosting
- XGBOOST

In [36]:
# Funciones generales
import sys
sys.path.append('../')
# A medida que avanzan las iteraciones se crearan mas funcinoes transversales
from funciones import *

# Tuneo de hiperparámetros
from sklearn.model_selection import ParameterGrid


Lectura de los datos resultantes de la iteración 2

In [37]:
import os 
# get current directory
path = os.getcwd()
# parent directory
path = os.path.abspath("../")
data_model = pd.read_csv(path + "/iteracion_2/datos_iteracion_2.csv")
data_model.head()

,precio,zona,barrio_sector,baños_familiares,area_bruta,numero_niveles,parqueaderos,alcobas_familiares,estrato,area_total,...,closet_de_linos,biblioteca,parqueadero_visitantes,gimnasio,piscina,salon_social,dispositivos_automatizacion,alarma,tipo_cocina_freq,tipo_pisos_freq
0,1100000.0,centro,castilla,1.0,95.0,1.0,0.0,3.0,0.0,95.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.861534,0.596709
1,950000.0,centro,el salvador,1.0,70.0,1.0,0.0,2.0,3.0,70.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.081652,0.596709
2,970000.0,centro,los angeles,1.0,38.0,1.0,0.0,1.0,4.0,38.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.081652,0.596709
3,1400000.0,centro,prado,1.0,50.0,1.0,1.0,2.0,0.0,50.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.861534,0.596709
4,800000.0,centro,12 de octubre,1.0,92.0,1.0,0.0,3.0,2.0,92.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.861534,0.596709


In [38]:
imprimir_dimensiones(data_model)

Numero de muestras: 2892, Número de columnas: 39


Eliminación de las variables por distribución y correlación (resultado de la iteración 2)

In [39]:
# Variables a descartar por correlación con variable final
columns_disc_corr = [
    "numero_niveles",
    "zona_ropas",
    "camaras_cctv",
    "cancha_polideportiva",
    "cancha_squash",
    "zona_bbq",
    "patio",
    "aire_acondicionado",
    "jacuzzi",
    "red_de_Gas",
    "terraza",
    "dispositivos_automatizacion",
    "alarma",
    "area_total",
    "porteria_24_7"
]

Por ahora no se usaran las variables de zona o barrio, por tanto, también se eliminan

In [40]:
columns_disc_corr.extend(['zona', 'barrio_sector'])

In [41]:
data_model = data_model.drop(columns=columns_disc_corr)
imprimir_dimensiones(data_model)

Numero de muestras: 2892, Número de columnas: 22


**Modelado**

Para cada tipo de modelo se realizara multiples ejecuciones que constan de la variación de hiperparámetros. Estos resultados serán tabulados y almacenados en un archivo csv identificado por cada tipo de modelo. Al final se presenta un resumen de los mejores resultados.

Las métricas evaluadas se ejecutaran en los conjuntos de entrenamiento y prueba con el fin de evitar el sobre entrenamiento, para esto se calcula la diferencia entre R2 entrnamiento y R2 prueba, esta no debe ser superior a 0.05

**Dvisión de los datos:** Entrenamiento 70%, Test 30%

Para esta ejecución se amplia el conjunto de datos de prueba

In [42]:
# Separación de las variables predictoras y la variable respuesta
X = data_model.drop(columns=['precio'])
Y = data_model['precio']

In [43]:
X_est = estandarizar(X)

In [44]:
x_train, x_test, y_train, y_test = train_test_split(X_est, Y, test_size = 0.30, train_size = 0.70, random_state = 17)
print("Tamaño de los Datos de Entrenamiento = ", x_train.shape)
print("Tamaño de los Datos de Validación = ", x_test.shape)
print("Tamaño del Vector de Clases de Entrenamiento = ", y_train.shape)
print("Tamaño del Vector de Clases de Prueba = ", y_test.shape)

Tamaño de los Datos de Entrenamiento =  (2024, 21)
Tamaño de los Datos de Validación =  (868, 21)
Tamaño del Vector de Clases de Entrenamiento =  (2024,)
Tamaño del Vector de Clases de Prueba =  (868,)


**Ada Boosting**

In [45]:
param_grid_ada = ParameterGrid({
    'n_estimators' : [30, 40, 50, 60, 80],
    'learning_rate' : [.1,.5, 1, 1.5],
    'estimator' : [LinearRegression(), DecisionTreeRegressor(), None]
})
# Ejecuciones del modelo
ejecutar_modelo(model = "AdaBoostRegressor", 
                x_train=x_train, 
                y_train=y_train, 
                x_test=x_test, 
                y_test=y_test, 
                params=param_grid_ada, 
                filename='AdaBoostRegressor')

Modelo: {'estimator': LinearRegression(), 'learning_rate': 0.1, 'n_estimators': 30}
Modelo: {'estimator': LinearRegression(), 'learning_rate': 0.1, 'n_estimators': 40}
Modelo: {'estimator': LinearRegression(), 'learning_rate': 0.1, 'n_estimators': 50}
Modelo: {'estimator': LinearRegression(), 'learning_rate': 0.1, 'n_estimators': 60}
Modelo: {'estimator': LinearRegression(), 'learning_rate': 0.1, 'n_estimators': 80}
Modelo: {'estimator': LinearRegression(), 'learning_rate': 0.5, 'n_estimators': 30}
Modelo: {'estimator': LinearRegression(), 'learning_rate': 0.5, 'n_estimators': 40}
Modelo: {'estimator': LinearRegression(), 'learning_rate': 0.5, 'n_estimators': 50}
Modelo: {'estimator': LinearRegression(), 'learning_rate': 0.5, 'n_estimators': 60}
Modelo: {'estimator': LinearRegression(), 'learning_rate': 0.5, 'n_estimators': 80}
Modelo: {'estimator': LinearRegression(), 'learning_rate': 1, 'n_estimators': 30}
Modelo: {'estimator': LinearRegression(), 'learning_rate': 1, 'n_estimators': 

**Gradient Boosting**

In [46]:
param_grid_gb = ParameterGrid({
    'learning_rate' : [.01,.1,.3],
    'max_depth' : [3,5,8],
    'max_features' : ['log2','sqrt'],
    'n_estimators' : [ 45,50, 100],
    'loss': ['squared_error', 'absolute_error', 'huber', 'quantile']
})
# Ejecuciones del modelo
ejecutar_modelo(model = "GradientBoostingRegressor", 
                x_train=x_train, 
                y_train=y_train, 
                x_test=x_test, 
                y_test=y_test, 
                params=param_grid_gb, 
                filename='GradientBoostingRegressor')

Modelo: {'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 3, 'max_features': 'log2', 'n_estimators': 45}
Modelo: {'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 3, 'max_features': 'log2', 'n_estimators': 50}
Modelo: {'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 3, 'max_features': 'log2', 'n_estimators': 100}
Modelo: {'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 3, 'max_features': 'sqrt', 'n_estimators': 45}
Modelo: {'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 3, 'max_features': 'sqrt', 'n_estimators': 50}
Modelo: {'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 3, 'max_features': 'sqrt', 'n_estimators': 100}
Modelo: {'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 5, 'max_features': 'log2', 'n_estimators': 45}
Modelo: {'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 5, 'max_features': 'log2', 'n_estimators': 50}
Modelo: {'learning_rate': 0.01, 'loss': 'squared_error', 'max_

**XGBOOST**

Para la ejecucion de este algoritmo es necesario otorgar el formato numérico a las columnas de entrenamiento y prueba

In [47]:
for column in x_train.columns:
    if column not in ['area_bruta', 'tipo_cocina_fre', 'tipo_pisos_freq']:
        x_train[column] = x_train[column].astype('float') 

for column in x_test.columns:
    if column not in ['area_bruta', 'tipo_cocina_fre', 'tipo_pisos_freq']:
        x_test[column] = x_test[column].astype('float') 

In [48]:
param_grid_xgb = ParameterGrid({
    'n_estimators' : [40, 80, 100],
    'max_depth' : [3,5],
    'learning_rate' : [.03, .05],
    'objective' : ['reg:squarederror'],
    'subsample' : [0.5, 0.8],
    'eta' : [.01, .05, .1],
})
# Ejecuciones del modelo
ejecutar_modelo(model = "XGBRegressor", 
                x_train=x_train, 
                y_train=y_train, 
                x_test=x_test, 
                y_test=y_test, 
                params=param_grid_xgb, 
                filename='XGBRegressor')

Modelo: {'eta': 0.01, 'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 40, 'objective': 'reg:squarederror', 'subsample': 0.5}
Modelo: {'eta': 0.01, 'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 40, 'objective': 'reg:squarederror', 'subsample': 0.8}
Modelo: {'eta': 0.01, 'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 80, 'objective': 'reg:squarederror', 'subsample': 0.5}
Modelo: {'eta': 0.01, 'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 80, 'objective': 'reg:squarederror', 'subsample': 0.8}
Modelo: {'eta': 0.01, 'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 100, 'objective': 'reg:squarederror', 'subsample': 0.5}
Modelo: {'eta': 0.01, 'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 100, 'objective': 'reg:squarederror', 'subsample': 0.8}
Modelo: {'eta': 0.01, 'learning_rate': 0.03, 'max_depth': 5, 'n_estimators': 40, 'objective': 'reg:squarederror', 'subsample': 0.5}
Modelo: {'eta': 0.01, 'learning_rate': 0.03, 'max_depth': 5, 'n_estimators

**Consolidación de mejores resultados:**

Dada las ejecuciones de los modelos con la variación de sus hiperparámetros, se consolida los mejores resultados obtenidos de cada algoritmo. Cabe resaltar que algunas combinaciones de parámetros tienen los mismos resultados en métricas.

**Ada boosting**

| R2_train | R2_test | RMSE_train | RMSE_test | MAPE_train | MAPE_test | Estimator | Learning rate | N estimators |
| --- | --- |  --- |  --- |  --- |  --- |  --- | --- | --- | 
| 0.73 | 0.71 | 1080592.66 | 1108223.99 | 0.3 | 0.32 |  | 0.1 | 30 |

**Gradient boosting**

| R2_train | R2_test | RMSE_train | RMSE_test | MAPE_train | MAPE_test | Learning rate | Loss | Max depth | Max features | N estimators |
| --- | --- |  --- |  --- |  --- |  --- |  --- | --- | --- |  --- |  --- | 
| 0.8 | 0.75 | 930445.85 | 1016608.13 | 0.18 | 0.21 | 0.1  | absolute_error | 5 | sqrt | 45 |

**XGBoost**

| R2_train | R2_test | RMSE_train | RMSE_test | MAPE_train | MAPE_test | Eta | Learning rate | Max depth | N estimators | Objetive | Subsample |
| --- | --- |  --- |  --- |  --- |  --- |  --- | --- | --- |  --- |  --- | --- |
| 0.72 | 0.7 | 1104313.47 | 1121524.01 | 0.21 | 0.21 | 0.05  | 0.05 | 3 | 40 | reg:squarederror | 0.5 |

**Conclusiones**

- Para estas ejecuciones se obtiene una mejora sustancial en las metricas del modelo para los algoritmos de __Gradient  boosting__ y __XG Boost__ obteniendo un MAPE a solo 0.3 puntos por encima de la métrica del negocio.
- Uniendo los resultados de la iteración anterior y la actual los algoritmos con mejores resultados son: __Gradient boosting__, __XG Boost__, __Random Forest__ y __SVR__, por tanto, en las iteraciones posteriores solo se ejecutaran estos algoritmos.
- Aunque la mejora en los algoritmos de boosting da un buen indicio del modelo a seleccionar, aún se debe obtener la métrica objetivo. Para esto, regresando a lo obtenido en los datos del scraping, se observaba que hay un desbalance en la cantidad de registros por zonas, siendo __Poblado__ la mas predominante, por lo tanto, se procederá a evaluar los mejores algoritmos en diferentes combiaciones de zonas y verificar si los datos de algunas de estas afectan los resultados obtenidos hasta ahora (este proceso se efectúa en la iteracion_5)